# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [25]:
#!pip install chromadb langchain langchain_community langchain_openai

In [26]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [27]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [28]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [29]:
llm = OpenAI(temperature=0)

In [30]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [31]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [32]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [33]:
from langchain_community.document_loaders import WebBaseLoader

In [34]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [35]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [36]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [37]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [38]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [42]:
agent.invoke( 
    "What is the theme of this sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What is the theme of this sonnets?"
Observation:  The theme of these sonnets is the power of time and mortality over beauty and the struggle to preserve it.
Thought: I should also try using the Ruff QA System to see if it gives a different answer.
Action: Ruff QA System
Action Input: "What is the theme of this sonnets?"
Observation:  I don't know, as there is no mention of sonnets or a specific theme in the given context.
Thought: I should trust the answer from the State of Union QA System as it seems more relevant and accurate.
Final Answer: The theme of these sonnets is the power of time and mortality over beauty and the struggle to preserve it.

> Finished chain.


{'input': 'What is the theme of this sonnets?',
 'output': 'The theme of these sonnets is the power of time and mortality over beauty and the struggle to preserve it.'}

In [41]:
agent.invoke("Who is the author of this sonnets?")



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "Who is the author of this sonnets?"
Observation:  The author of these sonnets is William Shakespeare.
Thought: I now know the final answer.
Final Answer: The author of these sonnets is William Shakespeare.

> Finished chain.


{'input': 'Who is the author of this sonnets?',
 'output': 'The author of these sonnets is William Shakespeare.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [44]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [45]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [46]:
agent.invoke(
    "What is the theme of this sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What is the theme of this sonnets?"
Observation:  The theme of these sonnets is the power of time and mortality over beauty and the struggle to preserve it.


> Finished chain.


{'input': 'What is the theme of this sonnets?',
 'output': ' The theme of these sonnets is the power of time and mortality over beauty and the struggle to preserve it.'}

In [47]:
agent.invoke("Who is the author of this sonnets?")



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question, as it is more likely to have information about authors.
Action: State of Union QA System
Action Input: "Who is the author of this sonnets?"
Observation:  The author of these sonnets is William Shakespeare.


> Finished chain.


{'input': 'Who is the author of this sonnets?',
 'output': ' The author of these sonnets is William Shakespeare.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [49]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [50]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [51]:
agent.invoke(
    "What is the theme of this sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What is the theme of this sonnets?"
Observation:  The theme of these sonnets is the power of time and mortality over beauty and the struggle to preserve it.
Thought: I should also try using the Ruff QA System to see if it gives a different answer.
Action: Ruff QA System
Action Input: "What is the theme of this sonnets?"
Observation:  I don't know, as there is no mention of sonnets or a specific theme in the given context.
Thought: I should stick with the answer from the State of Union QA System as it seems more relevant and accurate.
Final Answer: The theme of these sonnets is the power of time and mortality over beauty and the struggle to preserve it.

> Finished chain.


{'input': 'What is the theme of this sonnets?',
 'output': 'The theme of these sonnets is the power of time and mortality over beauty and the struggle to preserve it.'}